<h1 style='font-size:40px'> Tunagem de Hiperparâmetros</h1>

<h2 style='font-size:30px'> Otimização Bayesiana</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            A Otimização Bayesiana consiste na estimação da função-custo para um certo modelo. Seu objetivo é encontrar o conjunto de hiperparâmetros que retorne a melhor performance.
        </li>
        <li> 
            Inicialmente, combinações aleatórias de configurações são escolhidas para treinar e avaliar o modelo. Ao final, os ajustes que providenciaram os melhores desempenhos passam a ser considerados como promissores. Dessa forma, hiperparâmetros de valores próximos com os desses são feitos, sob a expectativa de que obtenhamos resultados ainda melhores!
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> O tradeoff entre Exploration e Exploitation</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            A Exploration consiste na mera exploração aleatória do espaço de hiperparâmetros. Por outro lado, a Exploitation é a exploração feita pelo modelo dos espaços considerados promissores para uma performance satisfatória. 
        </li>
        <li> 
            É muito importante equilibrarmos o foco que o modelo dará a ambas essas atividades. Um modelo com uma taxa de Exploration muito alta não aproveitará os espaços promissores, ocasionando em um underfitting. Por outro lado, uma Exploitation elevada fará com que o algoritmo não explore suficientemente o espaço de hiperparâmetros, correndo o risco de ter a sua performance presa no mínimo local.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Tunagem de Parâmetros com o KerasTuner</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O KerasTuner é uma biblioteca voltada à otimização de modelos Keras (apesar de também suportar os algoritmos do scikit-learn). Com ela, somos capazes de realizar Random Searches e Otimizações Bayesianas.
        </li>
    </ul>
</div>

In [51]:
# Vamos construir um modelo simples no Keras para ilustrarmos o uso do KerasTuner.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# O espaço em que os hiperparâmetros poderão ser inseridos é definido pela classe Hyperparameter.
from keras_tuner import HyperParameters

def build_model(hp):
    model = Sequential()
    
    # Inicializando o modelo com uma camada de Input.
    model.add(Input(shape=(None, 8)))
    
    # A quantidade de camadas é aleatória, de um a quatro.
    for i in range(hp.Int('n-hidden',min_value=1, max_value=4, step=1)):
        # Para cada hidden layer, o número de TLU's varia de 15 a 50, com uma diferença de, no mínimo, 10 unidades à cada iteração.
        # O modelo poderá ter uma activation function como 'relu' ou 'tanh'.
        model.add(Dense(hp.Int(f'units{i}',min_value=15, max_value=50, step=10),
                       activation=hp.Choice('activation', ['relu', 'tanh'])))
                  
    # Como faremos uma regressão, a camada de output deverá ter apenas um neurônio.
    model.add(Dense(1))
                  
    # Ajustando a learning rate do nosso otimizador.
    lr = hp.Float('learning_rate', min_value=.001, max_value=.3, sampling='log')
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
                  
    # A função deve retornar o modelo compilado.
    return model

<h3 style='font-size:30px;font-style:italic'> Random Search com o KerasTuner</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Ao invés de recorrermos ao Scikit-Learn, podemos performar uma Random Search com o KerasTuner. Isso remove a necessidade de termos que envelopar o modelo com um wrapper.
        </li>
    </ul>
</div>

In [58]:
from keras_tuner import RandomSearch

# 'objective' corresponde à métrica a ser monitorada para a consideração do melhor modelo.

# 'max_trials' é a quantidade de configurações distintas a serem criadas e testadas.

# 'executions_per_trial' equivale a quantos modelos serão criados e avaliados na mesma iteração. Isso serve para a redução do risco
# de uma inicialização ruim dos weights atrapalhar o desempenho do algoritmo.

# 'directory' e 'project_name' são os diretórios onde os registros das iterações serão anotados.

# 'overwrite' 
tuner = RandomSearch(build_model, objective='val_loss', max_trials=3, executions_per_trial=3, seed=42,
            directory='random_search', project_name='keras_rdn_search', overwrite=True)

# 'search_space_summary' disponibiliza um relatório sobre os espaços dos hiperparâmetros.
tuner.search_space_summary()

Search space summary
Default search space size: 4
n-hidden (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
units0 (Int)
{'default': None, 'conditions': [], 'min_value': 15, 'max_value': 50, 'step': 10, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.001, 'max_value': 0.3, 'step': None, 'sampling': 'log'}


In [61]:
# Vamos, brevemente, preparar os dados para treinamento, validação e teste.
from sklearn.
import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Carregando os dados e segregando os diferentes sets.
X,y = fetch_california_housing(return_X_y=True)
X_, X_test, y_, y_test = train_test_split(X,y, test_size=.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=.2, random_state=42)

# Modificando as escalas das features.
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [63]:
# Hora de realizarmos a nossa pesquisa de espaço. Para tornar o processo mais rápido, usaremos Early Stopping.
# 'search' recebe os mesmos argumentos do que o 'fit' de um modelo avulso.
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val), callbacks=[early_stopping])

Trial 3 Complete [00h 02m 13s]
val_loss: 0.33190874258677167

Best val_loss So Far: 0.33190874258677167
Total elapsed time: 00h 06m 55s
INFO:tensorflow:Oracle triggered exit


<p style='color:red'> Tópico "Start the search"</p>